In [2]:
from openpyxl import load_workbook
import flet as ft

def C(row: int, column: int, sheet):
    return sheet.cell(row, column).value

scfest = load_workbook('ESC.xlsx')

semifinal1, semifinal2 = scfest['SF1'], scfest['SF2']
voters = [C(1,i,semifinal1) for i in range(2,40)] # Voting countries
cnts1, cnts2 = [C(i,1,semifinal1) for i in range(2,18)], [C(i,1,semifinal2) for i in range(2,19)] # Participants of the First and Second Semi-Final respectively
points = [12,10] + [i for i in range(8,0,-1)]

sums1 = [sum(C(j,i,semifinal1) for j in range(2,18)) for i in range(2,40)]
sums2 = [sum(C(j,i,semifinal2) for j in range(2,19)) for i in range(2,40)]

ind,ch = 0,[]
def main(p: ft.Page):
    p.title = 'First Semi-Final: You Decide'
    p.window_width, p.window_height = 680, 800

    def disable(e):
        global ind, ch
        your_country.disabled = True
        if e.control.text == your_country.value:
            p.show_dialog(ft.AlertDialog(title=ft.Text('You cannot vote for yourself!')))
        else:
            if len(ch)==9:
                e.control.disabled = True
                p.add(ft.Text(value=f'{ind+1}. {e.control.text} - {points[ind]}'))
                ch.append(cnts1.index(e.control.text)+1)
                ind += 1
                count_votes()
                p.show_dialog(ft.AlertDialog(title=ft.Text('Thank you for voting!')))
            if len(ch)<9:
                e.control.disabled = True
                p.add(ft.Text(value=f'{ind+1}. {e.control.text} - {points[ind]}'))
                ch.append(cnts1.index(e.control.text)+1)
                ind += 1
        p.update()

    def change(e):
        if e.control.value != '':
            for i in bs:
                i.disabled = False
        p.update()

    bs = [ft.FilledButton(text=country, disabled=True, on_click=disable) for country in cnts1]
    rows = [ft.Row(controls=[bs[i],bs[i+(len(cnts1)//2)]]) for i in range(len(cnts1)//2)]
    your_country = ft.Dropdown(on_change=change, options=[ft.dropdown.Option(text=v) if sums1[i]<580 else ft.dropdown.Option(text=f'{v} (already voted 10 times)', disabled=True) for i,v in enumerate(voters)])

    def count_votes():
        for e,i in enumerate(ch):
            semifinal1.cell(row=i+1, column=voters.index(your_country.value)+2).value += points[e]
        scfest.save('ESC.xlsx')
        p.window_close()

    p.add(
        ft.Row(controls=[
            ft.Text(value='Choose your country: ', size=28),
            your_country
        ]),
    )
    for r in rows:
        p.add(r)
    # p.add(b17)

ft.app(main)